# Test de l'api Triflow (TRansient Instable Flow)

## Initialisation

### Imports

In [1]:
import logging
import itertools as it
from dill import copy

import pylab as pl
import seaborn as sns
import numpy as np
from scipy.signal import hamming

from triflow.solver import Solver
from triflow.helpers import init_4f_open
from triflow import displays, writers, materials, signals, schemes

%matplotlib inline

### logging

In [2]:
logger = logging.getLogger()
logger.handlers = []
handler = logging.StreamHandler()
formatter = logging.Formatter(
    '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.WARNING)

### Important après installation, mais peut être très long (~1h30)

from triflow.boundaries import openflow_boundary, periodic_boundary
from triflow.models.model_4fields import model
from triflow.models.model_full_fourrier import model as ffmodel
from triflow.make_routines import cache_routines_fortran
cache_routines_fortran(model, openflow_boundary, '4fields_open')
cache_routines_fortran(model, periodic_boundary, '4fields_per')
cache_routines_fortran(lambda: ffmodel(10), periodic_boundary, '10ff_per')
cache_routines_fortran(lambda: ffmodel(10), openflow_boundary, '10ff_open')

## Mise en place des paramètres physiques

In [3]:
parameters = {}

physical_parameters = materials.water()
physical_parameters['Beta'] = np.pi / 2
physical_parameters['Re'] = 15
physical_parameters['Bi'] = 2.81E-03
physical_parameters['freqd'] = 80
physical_parameters['signal_freq'] = physical_parameters['freq']
physical_parameters['signal_ampl'] = .5
parameters.update(physical_parameters)

initial_parameters = {
    'hini': 1.,
    'qini': 1 / 3,
    'sini': 0,
    'thetaini': physical_parameters['theta_flat'],
    'phiini': physical_parameters['phi_flat']
}
parameters.update(initial_parameters)

numerical_parameters = {'t': 0, 'dt': 1, 'tmax': 200,
                        'tol': 1E-1}
parameters.update(numerical_parameters)

domain_parameters = {'L': 200, 'Nx': 600}
parameters.update(domain_parameters)

parameters

{'B': 0.009994870185617077,
 'Beta': 1.5707963267948966,
 'Bi': 0.00281,
 'Ct': 6.12323399573677e-17,
 'Ka': 3363.38172912941,
 'L': 200,
 'Nx': 600,
 'Pe': 104.5,
 'Pr': 6.96666666666668,
 'Re': 15,
 'We': 265.8486656174358,
 'alpha': 36.09224577414277,
 'bhN': 0.0007007045269019624,
 'chi': 1.43540669856459e-07,
 'cp': 4180.0,
 'dt': 1,
 'freq': 0.011638206436051,
 'freqd': 80,
 'g': 9.81,
 'hN': 15.0,
 'hini': 1.0,
 'l_factor': 1427.13506422075,
 'l_v': 4.67136351267974e-05,
 'lamb': 0.6,
 'nu': 1e-06,
 'per': 85.923892611353,
 'perd': 0.0125,
 'phi_flat': -0.00989596133669492,
 'phiini': -0.00989596133669492,
 'qini': 0.3333333333333333,
 'rho': 1000,
 'sigma_inf': 0.072,
 'signal_ampl': 0.5,
 'signal_freq': 0.011638206436051,
 'sini': 0,
 't': 0,
 't_factor': 0.000145477580450637,
 't_v': 0.00218216370675956,
 'theta_flat': 0.990104038663305,
 'thetaini': 0.990104038663305,
 'tmax': 200,
 'tol': 0.1}

## Configuration du solveur

### Choix du solveur

In [4]:
solver, initial_fields = init_4f_open(parameters)

### Initialisation de la simulation

In [5]:
simul = solver.start_simulation(initial_fields, 0, **parameters)

### Ajout des perturbations en entrée

In [6]:
my_signal = (signals.ForcedSignal(simul, offset=1) +
             signals.BrownNoise(simul, noise_ampl=.01, fcut=.2))
simul.add_signal('h', my_signal)

### Choix des affichages et des writers

In [7]:
from triflow.misc import coroutine

## il est très simple de faire son propre affichage, c'est un générateur
# s'initialisant avec la simulation et qui yield les info voulues.
def simple_display_niter(simul):
    sdisplay = displays.simple_display(simul)
    while True:
        yield [*next(sdisplay), simul.internal_iter]
        
simul.set_display(simple_display_niter)

### Configuration "annexes"

## Calcul

In [8]:
running_simul = simul.copy()
running_simul.add_writer(writers.bokeh_nb_writer)
running_simul.conf['bokeh.fields'] = ['h', 'phi']
running_simul.conf['bokeh.fig.h'] = {'y_range': (0, 5)}
running_simul.conf['bokeh.fig'] = {'width': 600, 'height': 200}
for i, (t, field, niter) in enumerate(running_simul):
        print('\rt: %i, niter: %i' % (t, niter), end='')

t: 1, niter: 8

t: 200, niter: 2

In [15]:
running_simul = simul.copy()
running_simul.add_writer(writers.datreant_step_writer)
simul.conf['datreant.path'] = '.'
for i, (t, field, niter) in enumerate(running_simul):
    print('\rt: %i, niter: %i' % (running_simul.t, niter), end='')
print()
print(np.load(running_simul.save_path / 'data.npz'))
running_simul.save_path.rmtree()

t: 200, niter: 2
